<a href="https://colab.research.google.com/github/Mfr3ak/PyTorch-Practice/blob/PyTorch_Practice_Solutions/extras/exercises/01_pytorch_workflow_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01. PyTorch Workflow Exercise Template

The following is a template for the PyTorch workflow exercises.

It's only starter code and it's your job to fill in the blanks.

Because of the flexibility of PyTorch, there may be more than one way to answer the question.

Don't worry about trying to be *right* just try writing code that suffices the question.

You can see one form of [solutions on GitHub](https://github.com/mrdbourke/pytorch-deep-learning/tree/main/extras/solutions) (but try the exercises below yourself first!).

In [8]:
# Import necessary libraries
import torch
from torch import nn
import pandas as pd
import numpy as np

In [2]:
# Setup device-agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


## 1. Create a straight line dataset using the linear regression formula (`weight * X + bias`).
  * Set `weight=0.3` and `bias=0.9` there should be at least 100 datapoints total.
  * Split the data into 80% training, 20% testing.
  * Plot the training and testing data so it becomes visual.

Your output of the below cell should look something like:
```
Number of X samples: 100
Number of y samples: 100
First 10 X & y samples:
X: tensor([0.0000, 0.0100, 0.0200, 0.0300, 0.0400, 0.0500, 0.0600, 0.0700, 0.0800,
        0.0900])
y: tensor([0.9000, 0.9030, 0.9060, 0.9090, 0.9120, 0.9150, 0.9180, 0.9210, 0.9240,
        0.9270])
```

Of course the numbers in `X` and `y` may be different but ideally they're created using the linear regression formula.

In [5]:
# Create the data parameters
weight = 0.3
bias = 0.9


# Make X and y using linear regression feature
X = torch.arange(0,1,0.01)
y = weight * X +bias

sample_split = int(0.8*len(X))

print(f"Number of X samples: {len(X)}")
print(f"Number of y samples: {len(y)}")
print(f"First 10 X & y samples:\nX: {X[:10]}\ny: {y[:10]}")

Number of X samples: 100
Number of y samples: 100
First 10 X & y samples:
X: tensor([0.0000, 0.0100, 0.0200, 0.0300, 0.0400, 0.0500, 0.0600, 0.0700, 0.0800,
        0.0900])
y: tensor([0.9000, 0.9030, 0.9060, 0.9090, 0.9120, 0.9150, 0.9180, 0.9210, 0.9240,
        0.9270])


In [6]:
# Split the data into training and testing
X_train = X[:sample_split]
y_train = y[:sample_split]
X_test = X[sample_split:]
y_test = y[sample_split:]

In [ ]:
# Plot the training and testing data


## 2. Build a PyTorch model by subclassing `nn.Module`.
  * Inside should be a randomly initialized `nn.Parameter()` with `requires_grad=True`, one for `weights` and one for `bias`.
  * Implement the `forward()` method to compute the linear regression function you used to create the dataset in 1.
  * Once you've constructed the model, make an instance of it and check its `state_dict()`.
  * **Note:** If you'd like to use `nn.Linear()` instead of `nn.Parameter()` you can.

In [36]:
# Create PyTorch linear regression model by subclassing nn.Module
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(data = torch.randn(1,
                                                       requires_grad=True))
        self.bias = nn.Parameter(torch.randn(1,
                                requires_grad = True))
        # self.linear_layer = nn.Linear(in_features = 1,out_features = 1)
    def forward(self,X):
        return X*self.weights+self.bias

torch.manual_seed(42)
model_0 = LinearRegressionModel()
print(model_0.state_dict())



OrderedDict([('weights', tensor([0.3367])), ('bias', tensor([0.1288]))])


In [37]:
# Instantiate the model and put it to the target device
model_0.to(device)

LinearRegressionModel()

## 3. Create a loss function and optimizer using `nn.L1Loss()` and `torch.optim.SGD(params, lr)` respectively.
  * Set the learning rate of the optimizer to be 0.01 and the parameters to optimize should be the model parameters from the model you created in 2.
  * Write a training loop to perform the appropriate training steps for 300 epochs.
  * The training loop should test the model on the test dataset every 20 epochs.

In [38]:
# Create the loss function and optimizer
lossfn = nn.L1Loss()
optim = torch.optim.SGD(params = model_0.parameters(),lr = 0.01)

In [39]:
# Training loop
# Train model for 300 epochs
EPOCH = 300
torch.manual_seed(42)

# Send data to target device
X_train = X_train.to(device)
y_train = y_train.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)

for epoch in range(EPOCH):
  ### Training
  # Put model in train mode
  model_0.train()

  # 1. Forward pass
  y_pred = model_0(X_train)

  # 2. Calculate loss
  loss_train = lossfn(y_pred,y_train)

  # 3. Zero gradients
  optim.zero_grad()


  # 4. Backpropagation
  loss_train.backward()


  # 5. Step the optimizer
  optim.step()

  ### Perform testing every 20 epochs
  if epoch % 20 == 0:

    # Put model in evaluation mode and setup inference context
    model_0.eval()
    with torch.inference_mode():
      # 1. Forward pass
      y_pred_test = model_0(X_test)
      # 2. Calculate test loss
      loss_test = lossfn(y_pred_test,y_test)
      # Print out what's happening
      print(f"Epoch: {epoch} | Train loss: {loss_train} | Test loss: {loss_test}")

Epoch: 0 | Train loss: 0.7566978931427002 | Test loss: 0.7248174548149109
Epoch: 20 | Train loss: 0.5254928469657898 | Test loss: 0.4541124403476715
Epoch: 40 | Train loss: 0.2942880392074585 | Test loss: 0.18340764939785004
Epoch: 60 | Train loss: 0.07658576965332031 | Test loss: 0.07274086028337479
Epoch: 80 | Train loss: 0.05255373939871788 | Test loss: 0.11581540107727051
Epoch: 100 | Train loss: 0.04554273188114166 | Test loss: 0.10467890650033951
Epoch: 120 | Train loss: 0.038688670843839645 | Test loss: 0.08938358724117279
Epoch: 140 | Train loss: 0.03183453530073166 | Test loss: 0.07408786565065384
Epoch: 160 | Train loss: 0.024982361122965813 | Test loss: 0.0581042543053627
Epoch: 180 | Train loss: 0.018130186945199966 | Test loss: 0.04212063178420067
Epoch: 200 | Train loss: 0.011278990656137466 | Test loss: 0.02579306997358799
Epoch: 220 | Train loss: 0.004429381340742111 | Test loss: 0.009467745199799538
Epoch: 240 | Train loss: 0.00370177929289639 | Test loss: 0.0064894440

## 4. Make predictions with the trained model on the test data.
  * Visualize these predictions against the original training and testing data (**note:** you may need to make sure the predictions are *not* on the GPU if you want to use non-CUDA-enabled libraries such as matplotlib to plot).

In [49]:
# Make predictions with the model
y_pred_final = model_0(X_test)

In [50]:
# Plot the predictions (these may need to be on a specific device)


## 5. Save your trained model's `state_dict()` to file.
  * Create a new instance of your model class you made in 2. and load in the `state_dict()` you just saved to it.
  * Perform predictions on your test data with the loaded model and confirm they match the original model predictions from 4.

In [51]:
from pathlib import Path

# 1. Create models directory
Models_path = Path("models")

# 2. Create model save path
Models_path.mkdir(exist_ok = True)
Model_name = "01_trained_model.pth"
Model_save_path = Models_path/Model_name
# 3. Save the model state dict
torch.save(model_0.state_dict(),Model_save_path)

In [52]:
# Create new instance of model and load saved state dict (make sure to put it on the target device)
model_1 = LinearRegressionModel()
model_1.load_state_dict(torch.load(f=Model_save_path))
model_1 = model_1.to(device)

In [53]:
# Make predictions with loaded model and compare them to the previous
y_load_preds = model_1(X_test)
print(y_pred_final == y_load_preds)

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True], device='cuda:0')
